
![Ames Konut veri seti görüntüsü](https://storage.googleapis.com/kaggle-media/learn/images/lTJVG4e.png)


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Veriyi oku
X_full = pd.read_csv('data/train.csv', index_col='Id')
X_test_full = pd.read_csv('data/test.csv', index_col='Id')

# Eksik hedef verisi olan satırları çıkar, hedefi bağımsız değişkenlerden ayır
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# İşleri basit tutmak için sadece sayısal bağımsız değişkenleri kullanacağız
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Eğitim verisinden doğrulama setini ayır
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

Verilerin ilk beş satırını yazdırmak için bir sonraki kod hücresini kullanın.

In [2]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


İlk birkaç satırda birkaç eksik değer görebilirsiniz. Bir sonraki adımda, veri setindeki eksik değerler hakkında daha kapsamlı bir anlayış elde edeceksiniz.

# Adım 1: Ön araştırma


In [3]:
# Eğitim verisinin şekli (satır sayısı, sütun sayısı) nı bulun
print(X_train.shape)

# Eğitim verisindeki her bir sütunda eksik değerlerin sayısını bulun
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


### Bölüm A

Aşağıdaki soruları cevaplamak için yukarıdaki çıktıyı kullanın.

In [4]:
# Aşağıdaki satırı doldurun: Eğitim verisinde kaç satır var?
num_rows = 1168

# Aşağıdaki satırı doldurun: Eğitim verisinde kaç sütunda eksik değer var?
num_cols_with_missing = 3

# Aşağıdaki satırı doldurun: Tüm eğitim verisinde kaç eksik giriş var?
tot_missing = 212 + 6 + 58

### Bölüm B
Yukarıdaki cevaplarınızı göz önünde bulundurarak, eksik değerlerle başa çıkmanın en iyi yaklaşımının ne olduğunu düşünüyorsunuz?

Eksik değerlerle başa çıkmanın farklı yaklaşımlarını karşılaştırmak için, eğitimdeki aynı `score_dataset()` işlevini kullanacaksınız. Bu işlev, rastgele bir orman modelinden [ortalama mutlak hatayı](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) bildirir.

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Farklı yaklaşımları karşılaştırmak için fonksiyon
def score_dataset(X_train, X_valid, y_train, y_valid):
    # Rastgele Orman Regresyon modelini oluştur
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    
    # Modeli eğitim verisi ile eğit
    model.fit(X_train, y_train)
    
    # Doğrulama verisi üzerinde tahmin yap
    preds = model.predict(X_valid)
    
    # Tahminlerin Ortalama Mutlak Hata (MAE) değerini döndür
    return mean_absolute_error(y_valid, preds)

# Adım 2: Eksik değerlere sahip sütunları bırakın

Bu adımda, eksik değerlere sahip sütunları kaldırmak için `X_train` ve `X_valid` içindeki verileri önceden işleyeceksiniz. Önceden işlenmiş DataFrames'i sırasıyla `reduced_X_train` ve `reduced_X_valid` olarak ayarlayın.

In [6]:
# Aşağıdaki satırı doldurun: eksik değerleri olan sütunların adlarını alın
cols_with_missing = [col for col in X_train.columns
if X_train[col].isnull().any()]

# Aşağıdaki satırları doldurun: eğitim ve doğrulama verilerindeki sütunları bırakın
reduced_X_train = X_train.drop(cols_with_missing, axis=1) # eksik verileri buradan atıyoruz.
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

Bu yaklaşım için MAE'yi elde etmek amacıyla bir sonraki kod hücresini değişiklik yapmadan çalıştırın.

In [7]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


# Adım 3: Tahmin

### Bölüm A

Her sütun boyunca ortalama değerle eksik değerleri tahmin etmek için bir sonraki kod hücresini kullanın. Önceden işlenmiş Veri Çerçevelerini `imputed_X_train` ve `imputed_X_valid` olarak ayarlayın. Sütun adlarının `X_train` ve `X_valid` içindekilerle eşleştiğinden emin olun.

In [8]:
from sklearn.impute import SimpleImputer

# Aşağıdaki satırları doldurun: imputation (eksik değerleri doldurma)
# SimpleImputer'ı oluşturun
my_imputer = SimpleImputer()  
# Eğitim verisindeki eksik değerleri doldur ve veriyi dönüştür
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))  
# Doğrulama verisindeki eksik değerleri doldur ve veriyi dönüştür
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))  

# Aşağıdaki satırları doldurun: imputation sütun adlarını kaldırdı; geri koydu
# Eğitim verisi için sütun adlarını geri koy
imputed_X_train.columns = X_train.columns  
# Doğrulama verisi için sütun adlarını geri koy
imputed_X_valid.columns = X_valid.columns  

Bu yaklaşım için MAE'yi elde etmek amacıyla bir sonraki kod hücresini değişiklik yapmadan çalıştırın.

In [9]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


### Bölüm B

Her yaklaşımdan MAE'yi karşılaştırın. Sonuçlar hakkında sizi şaşırtan bir şey var mı? Bir yaklaşımın diğerinden daha iyi performans gösterdiğini neden düşünüyorsunuz?

# Adım 4: Test tahminleri oluşturun

Bu son adımda, eksik değerlerle başa çıkmak için seçtiğiniz herhangi bir yaklaşımı kullanacaksınız. Eğitim ve doğrulama özelliklerini önceden işledikten sonra, rastgele bir orman modelini eğitecek ve değerlendireceksiniz. Ardından, yarışmaya sunulabilecek tahminler oluşturmadan önce test verilerini önceden işleyeceksiniz!

### Bölüm A

Eğitim ve doğrulama verilerini önceden işlemek için bir sonraki kod hücresini kullanın. Önceden işlenmiş Veri Çerçevelerini `final_X_train` ve `final_X_valid` olarak ayarlayın. **Burada istediğiniz yaklaşımı kullanabilirsiniz!** Bu adımın doğru olarak işaretlenmesi için yalnızca şunları sağlamanız gerekir:
- önceden işlenmiş Veri Çerçevelerinin aynı sayıda sütuna sahip olması,
- önceden işlenmiş Veri Çerçevelerinin eksik değere sahip olmaması,
- `final_X_train` ve `y_train`'in aynı sayıda satıra sahip olması ve
- `final_X_valid` ve `y_valid`'in aynı sayıda satıra sahip olması.

In [10]:
# Ön işlenmiş eğitim ve doğrulama özellikleri
# Eksik değerleri median (ortanca) stratejisi ile doldurmak için SimpleImputer'ı oluştur
final_imputer = SimpleImputer(strategy="median") 
# Eğitim verisini dönüştür ve DataFrame olarak sakla
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))  
# Doğrulama verisini dönüştür ve DataFrame olarak sakla
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))  

Rastgele bir orman modelini eğitmek ve değerlendirmek için bir sonraki kod hücresini çalıştırın. (*Yukarıdaki `score_dataset()` fonksiyonunu kullanmadığımızı unutmayın, çünkü yakında eğitilmiş modeli test tahminleri üretmek için kullanacağız!*)

In [11]:
# Modeli tanımla ve eğit
model = RandomForestRegressor(n_estimators=100, random_state=0)  # Rastgele Orman Regresyon modelini oluştur
model.fit(final_X_train, y_train)  # Modeli, ön işlenmiş eğitim verisi ile eğit

# Doğrulama tahminlerini al ve MAE'yi hesapla
preds_valid = model.predict(final_X_valid)  # Doğrulama verisi üzerinde tahmin yap
print("MAE (Sizin yaklaşımınız):")  # Ortalama Mutlak Hata (MAE) değerini ekrana yazdır
print(mean_absolute_error(y_valid, preds_valid))  # MAE'yi hesapla ve ekrana yazdır

MAE (Sizin yaklaşımınız):
17791.59899543379


### Bölüm B

Test verilerinizi ön işlemek için bir sonraki kod hücresini kullanın. Eğitim ve doğrulama verilerini nasıl ön işlediğinizle uyuşan bir yöntem kullandığınızdan ve ön işlenmiş test özelliklerini `final_X_test` olarak ayarladığınızdan emin olun.

Ardından, `preds_test` içinde test tahminleri oluşturmak için ön işlenmiş test özelliklerini ve eğitilmiş modeli kullanın.

Bu adımın doğru olarak işaretlenmesi için yalnızca şunları sağlamanız gerekir:
- ön işlenmiş test Veri Çerçevesinde eksik değer yoktur ve
- `final_X_test` `X_test` ile aynı sayıda satıra sahiptir.

In [12]:
# Aşağıdaki satırı doldurun: test verisini ön işleme tabi tut
# Test verisini dönüştür ve DataFrame olarak sakla
final_X_test = pd.DataFrame(final_imputer.transform(X_test))  

# Aşağıdaki satırı doldurun: test tahminlerini al
# Test verisi üzerinde tahmin yap
preds_test = model.predict(final_X_test)  

kaydetmek için bir sonraki kod hücresini değişiklik yapmadan çalıştırın.

In [13]:
# Test tahminlerini dosyaya kaydet
output = pd.DataFrame({'Id': X_test.index,  # Tahmin edilen değerlerle birlikte ID'leri içeren bir DataFrame oluştur
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)  # DataFrame'i 'submission.csv' dosyasına kaydet, satır numaralarını dahil etme